In [1]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
import logging

# Ottieni il logger per 'yfinance' e imposta il suo livello a ERROR
yf_logger = logging.getLogger('yfinance')
yf_logger.setLevel(logging.ERROR)

model = fx.importa_modello("../Dati_borsa/Dati_addestramento/RF_est1000/Modello_RF_est1000.joblib")
model.verbose = 0

In [6]:
screener = pd.DataFrame(columns=["Categoria", "Ticker", "Borsa", "USIN", "Close", "Probabilità"])
tutti = pd.DataFrame(columns=["Categoria", "Ticker", "Borsa", "USIN", "Close", "Probabilità"])
elenco_ticker = fx.tickers_de_giro("..\\DeGiro con formule.xlsx")

i = 1
tot_tickers = len(elenco_ticker)
for _, ticker in elenco_ticker.iterrows():
    print(f"\r{i} di {tot_tickers}) Caricamento ticker {ticker['Ticker']}", end=' ', flush=True)
    try:
        df = yf.download(ticker["Ticker"], period="1y", interval="1d", progress=False)
        df = fx.crea_indicatori(df)
        df = fx.crea_altre_colonne(df)
        df = fx.elimina_valori_non_validi(df)
        df = fx.scala(df, ticker=ticker["Ticker"], percorso_dati="scalati/")
        df = fx.approssima(df)
        df.reset_index(inplace=True, drop=False)
        df = fx.create_lagged_features(df, lags=10)
        df = fx.approssima(df)
        df = fx.elimina_valori_non_validi(df)
    except Exception as e:
        print(e)
        continue
    if len(df) > 100:
        df = fx.definizione_parametri_e_target(df, n_barre_passate=10)
        X = df.drop(columns=["Target"])
        y = df["Target"]
        print(f"\r{i} di {tot_tickers}) Previsione ticker {ticker['Ticker']}", end=' ', flush=True)
        if len(X) > 0:
            X = X.iloc[-1:, :]
            y_proba_true = pd.Series(model.predict_proba(X)[-1, 1])
            tutti.loc[len(tutti)] = [ticker["Categoria"], ticker["Ticker"], ticker["Borsa"], ticker["USIN"], df.iloc[-1]["Close"], y_proba_true[0]]            
            if y_proba_true[0] >= 0.5:
                print(f"\n{i} di {tot_tickers}) Aggiunta ticker {ticker['Ticker']} allo screener", end='\n', flush=True)
                screener.loc[len(screener)] = [ticker["Categoria"], ticker["Ticker"], ticker["Borsa"], ticker["USIN"], df.iloc[-1]["Close"], y_proba_true[0]]
    i += 1

441 di 5220) Caricamento ticker AHL^PRC   


1 Failed download:
['AHL^PRC']: Exception('%ticker%: No data found, symbol may be delisted')


477 di 5220) Caricamento ticker ATCO^PRH 


1 Failed download:
['ATCO^PRH']: Exception('%ticker%: No data found, symbol may be delisted')


478 di 5220) Caricamento ticker ATCO^PRD 

...


1 Failed download:
['WBDR']: Exception('%ticker%: No data found, symbol may be delisted')


5219 di 5220) Caricamento ticker WIIT 


1 Failed download:
['WIIT']: Exception('%ticker%: No data found, symbol may be delisted')


5220 di 5220) Caricamento ticker ZV 


1 Failed download:
['ZV']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')


In [7]:
screener.to_excel("screener.xlsx")
tutti.to_excel("tutti.xlsx")
print("fine")        

fine
